In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import joblib

In [2]:
df = pd.read_parquet('/content/drive/MyDrive/GDM5 project/combined_data.parquet')

In [3]:
df

,name_file,time_stamp,asset_id,id,train_test,status_type_id,sensor_0_avg,sensor_1_avg,sensor_2_avg,wind_speed_3_avg,...,sensor_47,sensor_48,sensor_49,sensor_50,sensor_51,sensor_52_avg,sensor_52_max,sensor_52_min,sensor_52_std,sensor_53_avg
0,0.csv,03/08/2022 06:10,0,0,train,0,22.0,302.9,129.4,1.7,...,-496.0,0.0,0.0,-1280.0,-496.0,0.0,0.0,0.0,0.0,26.0
1,0.csv,03/08/2022 06:20,0,1,train,0,22.0,307.1,133.6,1.7,...,-490.0,0.0,0.0,-1278.0,-490.0,0.0,0.0,0.0,0.0,25.0
2,0.csv,03/08/2022 06:30,0,2,train,0,22.0,340.6,167.1,0.9,...,-490.0,0.0,0.0,-1356.0,-490.0,0.0,0.0,0.0,0.0,25.0
3,0.csv,03/08/2022 06:40,0,3,train,0,22.0,124.4,-49.1,1.5,...,-509.0,0.0,0.0,-1274.0,-509.0,0.0,0.0,0.0,0.0,26.0
4,0.csv,03/08/2022 06:50,0,4,train,0,22.0,66.2,-107.3,1.0,...,-499.0,0.0,0.0,-1284.0,-499.0,0.0,0.0,0.0,0.0,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1196742,92.csv,2023-04-16 09:20:00,11,54062,prediction,5,15.0,117.8,4.8,2.7,...,-328.0,0.0,0.0,-772.0,-328.0,1.4,2.7,0.0,1.1,18.0
1196743,92.csv,2023-04-16 09:30:00,11,54063,prediction,5,16.0,69.7,-32.0,2.3,...,-231.0,0.0,0.0,-725.0,-231.0,0.3,2.1,0.0,0.7,18.0
1196744,92.csv,2023-04-16 09:40:00,11,54064,prediction,5,16.0,69.8,-16.9,2.8,...,-492.0,0.0,0.0,-844.0,-492.0,1.9,2.1,1.6,0.1,19.0
1196745,92.csv,2023-04-16 09:50:00,11,54065,prediction,5,16.0,63.5,-10.6,2.4,...,-228.0,0.0,0.0,-731.0,-228.0,0.4,1.8,0.0,0.7,19.0


In [4]:
# how many values  in colunm asset_id ?
df['status_type_id'].value_counts()

,count
status_type_id,
0,898672
5,260387
3,23689
4,13999


In [5]:
#je veux créer une variable STATUS_ID qui vaut "normal" si la valeur de status_type_id vaut 0 ou 2 et "not normal" sinon
df['status_id'] = np.where(df['status_type_id'].isin([0, 2]), 'normal', 'not normal')

In [6]:
df['status_id'].value_counts()

,count
status_id,
normal,898672
not normal,298075


In [ ]:
sensor_columns = [col for col in df.columns if any(x in col for x in
                        ['sensor', 'wind_speed', 'power'])]
len(sensor_columns)

In [8]:
# Configuration de l'affichage
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("deep")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

In [9]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim=16):
        """
        Architecture de l'autoencoder pour la détection d'anomalies

        Args:
            input_dim: Dimension des données d'entrée (nombre de capteurs)
            encoding_dim: Dimension de la couche latente (compressée)
        """
        super(Autoencoder, self).__init__()

        # Encodeur - compresse les données d'entrée
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.2),  # Prévention du surapprentissage
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, encoding_dim),
            nn.ReLU()
        )

        # Décodeur - reconstruit les données à partir de la représentation compressée
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim),
            nn.Sigmoid()  # Normalise les sorties entre 0 et 1
        )

    def forward(self, x):
        """Propagation avant: encode puis décode les données"""
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [73]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim=16):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),  # Augmentation du nombre de neurones
            nn.LeakyReLU(),  # Utilisation de LeakyReLU
            nn.Dropout(0.3),  # Ajustement du taux de Dropout
            nn.Linear(128, 64),
            nn.LeakyReLU(),
            nn.Linear(64, encoding_dim),
            nn.LeakyReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 64),
            nn.LeakyReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 128),
            nn.LeakyReLU(),
            nn.Linear(128, input_dim),
            nn.Sigmoid()  # Vous pouvez changer cela si nécessaire
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [10]:
class WindTurbineAnomalyDetector:
    def __init__(self, encoding_dim=16):
        """
        Détecteur d'anomalies pour les données d'éoliennes

        Args:
            encoding_dim: Dimension de la couche latente de l'autoencoder
        """
        self.encoding_dim = encoding_dim
        self.scaler = StandardScaler()  # Pour normaliser les données
        self.model = None
        self.threshold = None
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Utilisation de: {self.device}")

    def preprocess_data(self, df):
        """
        Prétraitement des données: sélection et normalisation des capteurs

        Args:
            df: DataFrame contenant les données des capteurs

        Returns:
            X_scaled: Données normalisées
            sensor_columns: Noms des colonnes sélectionnées
        """
        # Sélection des colonnes pertinentes (capteurs uniquement)
        sensor_columns = [col for col in df.columns if any(x in col for x in
                        ['sensor', 'wind_speed', 'power'])]

        print(f"Nombre de capteurs sélectionnés: {len(sensor_columns)}")

        # Vérification et nettoyage des données
        X = df[sensor_columns].copy()

        # Remplacer les valeurs infinies par NaN
        X.replace([np.inf, -np.inf], np.nan, inplace=True)

        # Vérifier s'il y a des valeurs NaN et les remplacer par la moyenne de la colonne
        if X.isna().any().any():
            print(f"Attention: {X.isna().sum().sum()} valeurs NaN détectées et remplacées")
            X.fillna(X.mean(), inplace=True)

        # Normalisation des caractéristiques
        X_scaled = self.scaler.fit_transform(X)

        return X_scaled, sensor_columns

    def train(self, df, epochs=50, batch_size=64, learning_rate=0.0001,checkpoint_dir='checkpoints'):
        """
        Entraînement de l'autoencoder

        Args:
            df: DataFrame contenant les données d'entraînement
            epochs: Nombre d'époques d'entraînement
            batch_size: Taille des lots pour l'entraînement
            learning_rate: Taux d'apprentissage

        Returns:
            losses: Liste des pertes d'entraînement
        """
        # Prétraitement des données
        X, sensor_columns = self.preprocess_data(df)

        # Conversion en tenseurs PyTorch
        X_tensor = torch.FloatTensor(X).to(self.device)
        dataset = TensorDataset(X_tensor, X_tensor)  # Les entrées = sorties attendues
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        # Construction du modèle
        if self.model is None:
            self.model = Autoencoder(X.shape[1], self.encoding_dim).to(self.device)
            print(f"Modèle créé avec {X.shape[1]} entrées et dimension latente de {self.encoding_dim}")

        # Fonction de perte et optimiseur
        criterion = nn.MSELoss()  # Erreur quadratique moyenne
        optimizer = optim.Adam(self.model.parameters(), lr=learning_rate, weight_decay=1e-5)  # Ajout de régularisation L2

        # Créer le répertoire de checkpoints s'il n'existe pas
        Path(checkpoint_dir).mkdir(exist_ok=True)

        start_epoch = 0
        # Charger le dernier checkpoint s'il existe
        checkpoints = list(Path(checkpoint_dir).glob('epoch_*.pth'))
        if checkpoints:
            latest_checkpoint = max(checkpoints, key=lambda p: int(p.stem.split('_')[1]))
            checkpoint = torch.load(latest_checkpoint)
            start_epoch = checkpoint['epoch']
            self.model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            print(f"Reprise de l'entraînement à partir de l'epoch {start_epoch}")

        # Boucle d'entraînement
        losses = []
        self.model.train()  # Mode entraînement
        # Boucle d'entraînement
        losses = []
        self.model.train()  # Mode entraînement

        print("Début de l'entraînement...")
        for epoch in range(start_epoch,epochs):

            epoch_loss = 0
            for data, _ in dataloader:
                # Propagation avant
                outputs = self.model(data)
                loss = criterion(outputs, data)

                # Vérification de la perte
                if torch.isnan(loss):
                    print(f"Alerte: Perte NaN détectée à l'époque {epoch+1}. Arrêt de l'entraînement.")
                    return losses

                # Rétropropagation
                optimizer.zero_grad()  # Réinitialisation des gradients
                loss.backward()        # Calcul des gradients

                # Clip des gradients pour éviter l'explosion
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)

                optimizer.step()       # Mise à jour des poids

                epoch_loss += loss.item()

            # Calcul de la perte moyenne pour cette époque
            avg_loss = epoch_loss / len(dataloader)
            losses.append(avg_loss)

            print(f'Époque [{epoch+1}/{epochs}], Perte: {avg_loss:.6f}')
            # Enregistrer le checkpoint
            checkpoint_path = Path(checkpoint_dir) / f'epoch_{epoch + 1}.pth'
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': self.model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_loss,
            }, checkpoint_path)

        print("Entraînement terminé!")

        # Calcul du seuil d'erreur de reconstruction
        self.model.eval()  # Mode évaluation
        with torch.no_grad():
            reconstructions = self.model(X_tensor).cpu().numpy()

        # Calcul de l'erreur quadratique moyenne pour chaque exemple
        mse = np.mean(np.power(X - reconstructions, 2), axis=1)

        # Définition du seuil comme le 95ème percentile des erreurs
        self.threshold = np.percentile(mse, 95)
        print(f"Seuil d'anomalie calculé: {self.threshold:.6f}")

        return losses

    def detect_anomalies(self, df):
        """
        Détection des anomalies dans les données

        Args:
            df: DataFrame contenant les données à analyser

        Returns:
            results: DataFrame avec les erreurs de reconstruction et les anomalies
        """
        # Prétraitement des données
        X, _ = self.preprocess_data(df)

        # Conversion en tenseurs PyTorch
        X_tensor = torch.FloatTensor(X).to(self.device)

        # Obtention des reconstructions
        self.model.eval()
        with torch.no_grad():
            reconstructions = self.model(X_tensor).cpu().numpy()

        # Calcul de l'erreur MSE
        mse = np.mean(np.power(X - reconstructions, 2), axis=1)

        # Détection des anomalies (erreur > seuil)
        anomalies = mse > self.threshold

        # Création d'un DataFrame avec les résultats
        results = pd.DataFrame({
            'reconstruction_error': mse,
            'anomaly': anomalies
        }, index=df.index)

        return results

    def plot_history(self, losses):
        """Visualisation de l'historique d'entraînement"""
        plt.figure(figsize=(12, 4))
        plt.plot(losses)
        plt.title('Évolution de la perte pendant l\'entraînement')
        plt.xlabel('Époque')
        plt.ylabel('Perte (MSE)')
        plt.grid(True)
        plt.show()

    def plot_anomalies(self, df, results):
        """Visualisation des anomalies détectées"""
        plt.figure(figsize=(15, 7))

        # Tri par timestamp
        df_sorted = df.sort_values('time_stamp')
        results_sorted = results.loc[df_sorted.index]

        # Tracé de l'erreur de reconstruction
        plt.scatter(df_sorted['time_stamp'], results_sorted['reconstruction_error'],
                   c=results_sorted['anomaly'], cmap='coolwarm', alpha=0.7)

        plt.axhline(y=self.threshold, color='r', linestyle='--', label=f'Seuil ({self.threshold:.4f})')
        plt.title('Détection d\'anomalies dans les données d\'éoliennes')
        plt.xlabel('Date')
        plt.ylabel('Erreur de reconstruction')
        plt.colorbar(label='Anomalie')
        plt.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

        # Distribution des erreurs de reconstruction
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        sns.histplot(results['reconstruction_error'], bins=50, kde=True)
        plt.axvline(x=self.threshold, color='r', linestyle='--', label=f'Seuil ({self.threshold:.4f})')
        plt.title('Distribution des erreurs de reconstruction')
        plt.xlabel('Erreur de reconstruction')
        plt.ylabel('Fréquence')
        plt.legend()

        plt.subplot(1, 2, 2)
        sns.boxplot(x=results['anomaly'], y=results['reconstruction_error'])
        plt.title('Erreurs de reconstruction par classe')
        plt.xlabel('Anomalie')
        plt.ylabel('Erreur de reconstruction')
        plt.tight_layout()
        plt.show()

In [11]:
def analyze_sensor_contribution(detector, test_data, anomaly_results):
    """Analyse la contribution de chaque capteur aux anomalies détectées"""
    # Prétraitement des données
    X, sensor_columns = detector.preprocess_data(test_data)

    # Conversion en tenseurs PyTorch
    X_tensor = torch.FloatTensor(X).to(detector.device)

    # Obtention des reconstructions
    detector.model.eval()
    with torch.no_grad():
        reconstructions = detector.model(X_tensor).cpu().numpy()

    # Calcul de l'erreur par capteur
    sensor_errors = np.power(X - reconstructions, 2)

    # Création d'un DataFrame avec les erreurs par capteur
    sensor_error_df = pd.DataFrame(sensor_errors, columns=sensor_columns)

    # Ajout de l'information d'anomalie
    sensor_error_df['anomaly'] = anomaly_results['anomaly'].values

    # Calcul de l'erreur moyenne par capteur pour les anomalies vs non-anomalies
    anomaly_errors = sensor_error_df[sensor_error_df['anomaly']].mean()
    normal_errors = sensor_error_df[~sensor_error_df['anomaly']].mean()

    # Calcul du ratio d'erreur (anomalie / normal)
    error_ratio = anomaly_errors / normal_errors
    error_ratio = error_ratio.drop('anomaly')

    # Tri des capteurs par ratio d'erreur
    top_sensors = error_ratio.sort_values(ascending=False)

    return top_sensors, sensor_error_df


In [12]:
def run_analysis():
    # Chargement des données
    #data_path = Path(r'/content/drive/MyDrive/GDM5 project/combined_data.parquet')
    #df = pd.read_parquet(data_path)

    # Conversion du timestamp en format datetime
    df['time_stamp'] = pd.to_datetime(df['time_stamp'], format='mixed')

    # Séparation des données d'entraînement et de test
    train_data = df[(df['train_test'] == 'train') & (df['status_id'] == 'normal')] # Nouvelle ligne
    test_data = df[df['train_test'] == 'prediction']

    print(f"\nDonnées d'entraînement: {len(train_data)} lignes")
    print(f"Données de test: {len(test_data)} lignes")

    # Création et entraînement du détecteur
    detector = WindTurbineAnomalyDetector(encoding_dim=16)
    losses = detector.train(train_data, epochs=20, checkpoint_dir='analysis_checkpoints')
    # Visualisation de l'entraînement
    detector.plot_history(losses)

    # Détection des anomalies
    print("Détection des anomalies sur les données de test...")
    anomaly_results = detector.detect_anomalies(test_data)

    # Statistiques des anomalies
    anomaly_count = anomaly_results['anomaly'].sum()
    print(f"Nombre d'anomalies détectées: {anomaly_count}")
    print(f"Pourcentage d'anomalies: {(anomaly_count / len(test_data)) * 100:.2f}%")

    # Visualisation des anomalies
    detector.plot_anomalies(test_data, anomaly_results)

    # Analyse des capteurs les plus influents
    top_sensors, sensor_error_df = analyze_sensor_contribution(detector, test_data, anomaly_results)

    # Visualisation des capteurs les plus influents
    plt.figure(figsize=(14, 6))
    top_sensors.head(15).plot(kind='bar')
    plt.title('Top 15 des capteurs contribuant le plus aux anomalies')
    plt.xlabel('Capteur')
    plt.ylabel('Ratio d\'erreur (Anomalie/Normal)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Sauvegarde du modèle
    model_path = Path(r'c:\Users\moham\Desktop\GDM5 project\models')
    model_path.mkdir(exist_ok=True)
    torch.save(detector.model.state_dict(), model_path / 'autoencoder_model.pth')
    joblib.dump(detector.scaler, model_path / 'scaler.pkl')
    joblib.dump(detector.threshold, model_path / 'threshold.pkl')

    return detector, anomaly_results, top_sensors

if __name__ == "__main__":
    # Exécuter l'analyse complète
    detector, anomaly_results, top_sensors = run_analysis()


Données d'entraînement: 859574 lignes
Données de test: 50593 lignes
Utilisation de: cuda
Nombre de capteurs sélectionnés: 81
Attention: 10 valeurs NaN détectées et remplacées
Modèle créé avec 81 entrées et dimension latente de 16
Début de l'entraînement...
Époque [1/20], Perte: 0.671420
Époque [2/20], Perte: 0.632547
Époque [3/20], Perte: 0.624781
Époque [4/20], Perte: 0.621608
Époque [5/20], Perte: 0.619170
Époque [6/20], Perte: 0.617152
Époque [7/20], Perte: 0.615400
Époque [8/20], Perte: 0.612613
Époque [9/20], Perte: 0.611059
Époque [10/20], Perte: 0.610120
Époque [11/20], Perte: 0.609382
Époque [12/20], Perte: 0.608725
Époque [13/20], Perte: 0.608214
Époque [14/20], Perte: 0.607721


KeyboardInterrupt: 

In [78]:
import os
import shutil

checkpoint_dir = 'analysis_checkpoints'  # Remplacez par le répertoire approprié

for filename in os.listdir(checkpoint_dir):
    if filename.startswith('epoch_') and filename.endswith('.pth'):
        file_path = os.path.join(checkpoint_dir, filename)
        os.remove(file_path)

# Ou, pour supprimer tout le contenu du répertoire :
# shutil.rmtree(checkpoint_dir)  # Attention, cela supprimera tout dans le répertoire !